In [71]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import time
from sklearn import svm
import time
from sklearn.model_selection import KFold

In [2]:
def tpr_weight_funtion(y_true,y_predict):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3

In [5]:
# 加载训练集
opr_train = pd.read_csv('../python_env01/data/new/operation_train_new.csv')
tra_train = pd.read_csv('../python_env01/data/new/transaction_train_new.csv')
tag = pd.read_csv('../python_env01/data/new/tag_train_new.csv', index_col='UID')

# 加载测试集
opr_test = pd.read_csv('../python_env01/data/new/operation_round1_new.csv')
tra_test = pd.read_csv('../python_env01/data/new/transaction_round1_new.csv')

# UID防重叠
opr_test['UID'] = opr_test['UID'] + 40000
tra_test['UID'] = tra_test['UID'] + 40000

/home/liao/code/python_env01/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [80]:
print('shape of train, opr:%s, tra:%s' % (str(opr_train.shape), str(tra_train.shape)))
print('shape of test, opr:%s, tra:%s' % (str(opr_test.shape), str(tra_test.shape)))

shape of train, opr:(1460843, 22), tra:(264654, 30)
shape of test, opr:(1769049, 20), tra:(168981, 27)


In [6]:
# UID防重叠
opr_test['UID'] = opr_test['UID'] + 40000
tra_test['UID'] = tra_test['UID'] + 40000

In [7]:
test_id = pd.concat([opr_test['UID'], tra_test['UID']])
train_id = pd.concat([opr_train['UID'], tra_train['UID']])
print('train, min:%s, max:%s, len:%s' % (train_id.min(), train_id.max(), train_id.unique().size))
print('test   min:%s, max:%s, len:%s' % (test_id.min(), test_id.max(), test_id.unique().size))

train, min:10000, max:67756, len:31179
test   min:70000, max:127763, len:31198


## 操作特征

In [8]:
def deal_opr(opr, file_name):
    start = time.time()
    train = pd.DataFrame()
    train['day_nunique'] = opr.groupby('UID')['day'].nunique()
    train['record_count'] = opr.groupby('UID').count()['day']
    train['opr_fre'] = train['record_count'] / train['day_nunique']
    train['mac2_num'] = opr.groupby('UID')['mac2'].nunique()
    train['geo_num'] = opr.groupby('UID')['geo_code'].nunique()
    # mode
    '''c8741ce15ceac2a4
    d25caee90b27fa9b
    79f8b86938180c3c
    acfaded7e04e7ba0
    b501fa4fc58206b9
    072eee5c88d380df
    00094ae2a1d62504
    20a91b45ef8f8221
    cd75f033b0488ace'''
    #mode = opr[['UID','mode','success','time','os','device2']]
    opr.loc[:,'mode_code'] = opr['mode'].map({'c8741ce15ceac2a4':1, 
                            'd25caee90b27fa9b':2,
                            '79f8b86938180c3c':3,
                            'acfaded7e04e7ba0':4,
                            'b501fa4fc58206b9':5,
                            '072eee5c88d380df':6,
                            '00094ae2a1d62504':7,
                            '20a91b45ef8f8221':8,
                            'cd75f033b0488ace':9,
                           })
    opr['mode_code'] = opr['mode_code'].fillna(0).astype(int)
    mode_type = opr['mode_code'].unique()
    mode_type.sort()
    # 设置mode name
    for i in mode_type:
        train['mode_%s' % i] = 0

    # 设置time name
    for i in range(0, 24):
        train['time_%02d' % i] = 0
    # 设置os name
    for i in opr['os'].value_counts().index.values:
        train['os_%s' % i] = 0
    # 设置phone name
    phone_name = ['IPHONE', 'VIVO', 'OPPO', 'HUAWEI', 'MI', 'REDMI', 'MHA-AL00', 'HM', 'ZTE', 'other']
    phone_map = {None:-1,'IPHONE':1, 'VIVO':2, 'OPPO':3, 'HUAWEI':4, 'MI':5, 'REDMI':6, 'MHA-AL00':7, 'HM':8, 'ZTE':9}
    opr.loc[:, 'phone'] = opr.loc[:, 'device2'].str.split(n=1, expand=True).iloc[:, 0].map(phone_map)
    opr['phone'] = opr['phone'].fillna(0).astype(int)
    #display(opr.loc[:10, ['device2','phone']])
    phone_type = opr['phone'].unique()
    phone_type.sort()
    for i in phone_type:
        train['phone_%s' % int(i)] = 0


    for name,group in opr.groupby('UID'):
        # 各种操作的记录数
        model_vc = group['mode_code'].value_counts()
        mode_col_name = ['mode_%s' % i for i in model_vc.index]
        train.loc[name, mode_col_name] = model_vc.values

        # 操作成功的记录数
        train.loc[name, 'success_num'] = group['success'].sum()

        # 各小时的操作数
        time_vc = group['time'].str[:2].value_counts()
        time_col_name = ['time_%02d' % int(i) for i in time_vc.index]
        train.loc[name, time_col_name] = time_vc.values

        # 操作系统数
        os_vc = group['os'].value_counts()
        os_col_name = ['os_%s' % i for i in os_vc.index.values]
        train.loc[name, os_col_name] = os_vc.values

        # phone name
        # 这个字段要先预处理，把不关心的全部转换
        phone = group.loc[group['phone'] >= 0, 'phone']
        phone_vc = phone.value_counts()
        phone_col_name = ['phone_%s' % i for i in phone_vc.index]
        train.loc[name, phone_col_name] = phone_vc.values

        #break

    #mode.groupby.value_counts()
    train = train.fillna(0).astype(int)
    #display(train)
    train.to_csv(file_name)
    print('shape ', train.shape)
    end = time.time
    #print('total time:%s', (end - start))

## 交易特征

In [9]:

def deal_trans(tra, file_name):
    start = time.time()
    tra_fea = pd.DataFrame()
    # set channel name
    for i in tra['channel'].value_counts().index.values:
        tra_fea['channel_%s' % i] = 0
    # set day name
    day_type= tra['day'].value_counts().index.values
    day_type.sort()
    for i in day_type:
        tra_fea['day_%s' % i] = 0
    # set amt src1
    amt_src1_map = {
    '155c9e1c32bd0fa2':1,
    'c5fc631370cabc0d':2,
    '4d7831c6f695ab19':3,
    'f29829bc82459191':4,
    '9451ef3c5a0d6807':5,
    'acdbdb842ac20f1e':6,
    'a571c7fda8b7df37':7,
    '992d3ce08a4ca702':8,
    '27c42480134c0d02':9,
    '8c9987909b3e95a4':10,
    }
    tra.loc[:, 'amt_src1_map'] = tra.loc[:, 'amt_src1'].map(amt_src1_map)
    tra['amt_src1_map'] = tra['amt_src1_map'].fillna(0).astype(int)
    #display(opr.loc[:10, ['device2','phone']])
    amt_src1_map_type = tra['amt_src1_map'].unique()
    amt_src1_map_type.sort()
    for i in amt_src1_map_type:
        tra_fea['amt_src1_%s' % int(i)] = 0
    # set merchant
    merchant_map = {
    '3bd5cf7c40962299':1,
    'e36d1d861d5fc9ec':2,
    'd8babe2d19fa0c08':3,
    '1f72814f76a984fa':4,
    'c71c876b8979028e':5,
    '3f6d3d0f42519ea4':6,
    '8f57527418b3f457':7,
    '0e90f47392008def':8,
    'fa18843382c90848':9,
    'f25bb3986a5a1d6d':10,
    '5776870b5747e14e':11,
    '0bdfe811a0e60e0f':12,
    'e1bad078cf58a817':13,
    'ad0aa55c2ea8ff76':14,
    '1e70ea89a4cbb3fe':15
    }
    tra.loc[:, 'merchant_map'] = tra.loc[:, 'merchant'].map(merchant_map)
    tra['merchant_map'] = tra['merchant_map'].fillna(0).astype(int)
    merchant_map_type = tra['merchant_map'].unique()
    merchant_map_type.sort()
    for i in merchant_map_type:
        tra_fea['merchant_%s' % i] = 0
    # set trans_type1
    trans_type1_map = {
    'c2f2023d279665b2':1,
    '6d55c54c8b1056fb':2,
    '61bfb66c928f36ac':3,
    '26bcf43a19df14c8':4,
    'e0d7b8768da99dd4':5,
    'd9c417304a5ae70c':6,
    '9d7dd7b80e806024':7,
    'a19e7a8951e54c06':8,
    'ced62357ad496957':9,
    'eb8d10591677bbe1':10,
    '4adc3de71fe1a83c':11,
    '85bced5214d33ad2':12,
    'e903cf2a79b83d37':13,
    'fd4d2d1006a95637':14,
    '3f469aa3836e71cb':15,
    }
    tra.loc[:, 'trans_type1_map'] = tra.loc[:, 'trans_type1'].map(trans_type1_map)
    tra['trans_type1_map'] = tra['trans_type1_map'].fillna(0)
    trans_types1_type = tra['trans_type1_map'].unique()
    trans_types1_type.sort()
    for i in trans_types1_type:
        tra_fea['trans_types1_%s' % int(i)] = 0
    # set trans_type2
    tra['trans_type2'] = tra['trans_type2'].fillna(0)
    trans_types2_type = tra['trans_type2'].unique()
    trans_types2_type.sort()
    for i in trans_types2_type:
        tra_fea['trans_types2_%s' % int(i)] = 0
    # set market_type
    tra['market_type'] = tra['market_type'].fillna(0)
    market_type_type = tra['market_type'].unique()
    market_type_type.sort()
    for i in market_type_type:
        tra_fea['market_type_%s' % int(i)] = 0

    # 生成特征
    for name,group in tra.groupby('UID'):
        # 交易平台
        channel_vc = group['channel'].value_counts()
        channel_col_name = ['channel_%s' % i for i in channel_vc.index.values]
        tra_fea.loc[name, channel_col_name] = channel_vc.values
        # 交易记录
        tra_fea.loc[name, 'tra_count'] = group.shape[0]
        # 交易日期
        day_vc = group['day'].value_counts()
        day_col_name = ['day_%s' % i for i in day_vc.index.values]
        tra_fea.loc[name, day_col_name] = day_vc.values
        # 交易总额
        tra_fea.loc[name, 'trans_amt_sum'] = group['trans_amt'].sum()
        # 交易额标准差
        tra_fea.loc[name, 'trans_amt_std'] = group['trans_amt'].std()
        # 资金类型1
        amt_src1_vc = group['amt_src1_map'].value_counts()
        as_col_name = ['amt_src1_%s' % i for i in amt_src1_vc.index.values]
        tra_fea.loc[name, as_col_name] = amt_src1_vc.values
        # 商户标识
        merchant_vc = group['merchant_map'].value_counts()
        merchant_col_name = ['merchant_%s' % i for i in merchant_vc.index.values]
        tra_fea.loc[name, merchant_col_name] = merchant_vc.values
        # 交易类型1
        trans_type1_vc = group['trans_type1_map'].value_counts()
        trans_type1_col_name = ['trans_types1_%s' % int(i) for i in trans_type1_vc.index.values]
        tra_fea.loc[name, trans_type1_col_name] = trans_type1_vc.values
        # 交易类型2
        trans_type2_vc = group['trans_type2'].value_counts()
        trans_type2_col_name = ['trans_types2_%s' % int(i) for i in trans_type2_vc.index.values]
        tra_fea.loc[name, trans_type2_col_name] = trans_type2_vc.values
        # 营销活动类型
        market_type_vc = group['market_type'].value_counts()
        market_type_col_name = ['market_type_%s' % int(i) for i in market_type_vc.index.values]
        tra_fea.loc[name, market_type_col_name] = market_type_vc.values
        #break

    tra_fea.index.name = 'UID'
    tra_fea = tra_fea.fillna(0)
    tra_fea = tra_fea.astype('int')
    tra_fea.to_csv(file_name)
    print('shape of tra fea:', tra_fea.shape)
    end = time.time()
    #print('total time:%s' % (end - start))

## 新版预处理函数

In [74]:
def str2num(x):
    return x
    
opr_train.iloc[:10, :].loc[:, ['UID','day','os']].groupby('UID')

In [79]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                              'foo', 'bar', 'foo', 'foo'],
                       'B' : ['one', 'one', 'two', 'three',
                              'two', 'two', 'one', 'three'],
                     'C' : np.random.randn(8),
                      'D' : np.random.randn(8),
                   'E' : np.random.randn(8)
                  })

def get_letter_type(letter):
        if letter.lower() in 'aeiou':
            return 'vowel'
        else:
            return 'consonant'

for name, grouped in df.groupby(get_letter_type, axis=1):
    print(name)
    print(grouped)

consonant
       B         C         D
0    one -2.177233  0.020503
1    one -0.473083 -0.933264
2    two  0.499981  0.419310
3  three  0.655119 -0.665449
4    two  0.318130  1.684115
5    two  0.147339  1.256607
6    one -1.267585 -0.141239
7  three -2.271606 -0.226425
vowel
     A         E
0  foo  1.386626
1  bar  0.976582
2  foo  0.376581
3  bar  0.257805
4  foo -1.126475
5  bar  1.671087
6  foo -0.525841
7  foo -0.158737


## 生成特征

In [10]:
# 操作数据
# 处理全部数据
# opr_all = pd.concat([opr_test, opr_train])
# print('opr all:')
# opr_all_file_name = '../python_env01/data/operation_feature01_train_test.csv'
# deal_opr(opr_all, opr_all_file_name)
# 处理训练集
print('opr train:')
opr_train_file_name = '../python_env01/data/operation_feature01_train.csv'
deal_opr(opr_train, opr_train_file_name)
print('opr test:')
opr_test_file_name = '../python_env01/data/operation_feature01_test.csv'
deal_opr(opr_test, opr_test_file_name)

# 交易数据
# 处理全部数据
# print('tra all:')
# tra_all = pd.concat([tra_test, tra_train])
# all_tra_file_name = '../python_env01/data/transaction_feature01_train_test.csv'
# deal_trans(tra_all, all_tra_file_name)
# 训练集
print('tra train:')
train_tra_file_name = '../python_env01/data/transaction_feature01_train.csv'
deal_trans(tra_train, train_tra_file_name)
print('tra test:')
tra_test_file_name = '../python_env01/data/transaction_feature01_test.csv'
deal_trans(tra_test, tra_test_file_name)

opr train:
shape  (29728, 58)
opr test:
shape  (27887, 58)
tra train:
shape of tra fea: (30542, 88)
tra test:
shape of tra fea: (30101, 86)


## 加载特征数据

In [26]:
trans_train = pd.read_csv('../python_env01/data/transaction_feature01_train.csv', index_col='UID')
oper_train = pd.read_csv('../python_env01/data/operation_feature01_train.csv', index_col='UID')
data_train = pd.concat([trans_train, oper_train], axis=1)

trans_test = pd.read_csv('../python_env01/data/transaction_feature01_test.csv', index_col='UID')
oper_test = pd.read_csv('../python_env01/data/operation_feature01_test.csv', index_col='UID')
data_test = pd.concat([trans_test, oper_test], axis=1)

# 加载验证集数据
#trans_all = pd.read_csv('../python_env01/data/transaction_feature01_train_test.csv', index_col='UID')
#oper_all = pd.read_csv('../python_env01/data/operation_feature01_train_test.csv', index_col='UID')
#data_all = pd.concat([trans_all, oper_all], axis=1)

In [28]:
print('shape of train:%s, oper:%s' % (str(trans_train.shape), str(oper_train.shape)))
print('shape of train:', data_train.shape)

print('shape of test:%s, %s' % (str(trans_test.shape), str(oper_test.shape)))
print('shape of test:', data_test.shape)

# print('shape of all:%s, oper:%s' % (str(trans_all.shape), str(oper_all.shape)))
# print('shape of all data:', data_all.shape)

shape of train:(30542, 88), oper:(29728, 58)
shape of train: (31179, 146)
shape of test:(30101, 86), (27887, 58)
shape of test: (31198, 144)


In [30]:
# 训练集比全部数据集少了两个字段
for name in data_train.columns.values:
    if name not in data_test.columns.values:
        print('del %s on train' % name)
        data_train = data_train.drop(name, axis=1)
print('***'*8)
for name in data_test.columns.values:
    if name not in data_train.columns.values:
        print('del %s on test' % name)
        data_test = data_test.drop(name, axis=1)
        
print('###' * 8)
print('shape of train: ', data_train.shape)
print('shape of test:', data_test.shape)

************************
########################
shape of train:  (31179, 142)
shape of test: (31198, 142)


In [14]:
#data2 = data.dropna()
# trans_types1_nan 也是全0
del_col_name = ['trans_types1_0', 'day_31']
for name in del_col_name:
    if name in data_train.columns.values:
        data_train = data_train.drop(name, axis=1)
data_train = data_train.drop('phone_-1', axis=1)
print('shape of data_train:', data_train.shape)

shape of data_train: (31179, 145)


In [15]:
del_col_name = ['trans_types1_0', 'day_31']
for name in del_col_name:
    if name in data_test.columns.values:
        data_test = data_test.drop(name, axis=1)
data_test = data_test.drop('phone_-1', axis=1)
print('shape of data_test:', data_test.shape)

shape of data_test: (31198, 141)


In [16]:
data_train = (data_train - data_train.min()) / (data_train.max() - data_train.min())
data_test = (data_test - data_test.min()) / (data_test.max() - data_test.min())


In [17]:
print('train, min:%s, max:%s, len:%s' % (train.index.min(), train.index.max(), str(train.shape)))
print('test, min:%s, max:%s, len:%s' % (test.index.min(), test.index.max(), str(test.shape)))

NameError: name 'train' is not defined

In [91]:
tag.shape

(31179, 1)

## 建模

In [54]:
data_train1 = (data_train - data_train.min()) / (data_train.max() - data_train.min())
data_train1.loc[data_train1.iloc[:, 1].isna()]

,channel_140,channel_102,channel_118,channel_119,channel_106,day_1,day_2,day_3,day_4,day_5,...,phone_2,phone_3,phone_4,phone_5,phone_6,phone_7,phone_8,phone_9,success_num,y
UID,,,,,,,,,,,,,,,,,,,,,
10062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.010178,0.0,0.005217,1.0
10102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.001304,1.0
10141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.024038,0.000000,0.0,0.003913,0.0
10194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.002415,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.001304,0.0
10262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.007634,0.0,0.001739,1.0
10293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.010178,0.0,0.005217,1.0
10316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.012723,0.0,0.005652,1.0
10347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.002174,1.0
10361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000870,1.0


In [76]:
# LR 训练数据建模
data_train1 = (data_train - data_train.min()) / (data_train.max() - data_train.min())
data_train1['y'] = tag
data_train2 = data_train1.fillna(0)
kf = KFold(n_splits=10)

#X_train,X_test,y_train,y_test = train_test_split(data_train2.iloc[:, :-1], 
#        data_train2.iloc[:, -1],test_size=0.5,random_state=0)

score = []
for train_index, test_index in kf.split(data_train2):
    X_train = data_train2.iloc[train_index, :-1]
    X_test = data_train2.iloc[test_index, :-1]
    y_train = data_train2.iloc[train_index, -1]
    y_test = data_train2.iloc[test_index, -1]

    start = time.time()
    lr_clf = LogisticRegression(random_state=0, solver='lbfgs',
                                multi_class='multinomial', class_weight={0:1,1:10})
    lr_clf.fit(X_train, y_train)
    y_pred = lr_clf.predict_proba(X_test)
    s = tpr_weight_funtion(y_test, y_pred[:, 1])
    print('TPR:%s' % s)
    score.append(s)
    end = time.time()
    print('total time:%-2f' % (end - start))
    
ss = pd.Series(score)
print('score mean:', ss.mean())

/home/liao/code/python_env01/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


TPR:0.4320987654320988
total time:2.250762


/home/liao/code/python_env01/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


TPR:0.38189415041782726
total time:2.126040


/home/liao/code/python_env01/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


...

/home/liao/code/python_env01/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


TPR:0.3212678936605317
total time:2.401561


/home/liao/code/python_env01/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


TPR:0.42016293279022404
total time:2.080386
TPR:0.36461538461538456
total time:2.212680
score mean: 0.38923111549829714


/home/liao/code/python_env01/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [80]:
data_train.loc[data_train.iloc[:, 1].isna()]

,channel_140,channel_102,channel_118,channel_119,channel_106,day_1,day_2,day_3,day_4,day_5,...,phone_2,phone_3,phone_4,phone_5,phone_6,phone_7,phone_8,phone_9,success_num,y
UID,,,,,,,,,,,,,,,,,,,,,
10062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,12.0,1
10102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1
10141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,9.0,0
10194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0
10262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,1
10293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,12.0,1
10316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,13.0,1
10347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1
10361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1


In [81]:
# LR 数据预测
data_train0 = data_train.dropna(how='all')
data_train1 = (data_train0 - data_train0.min())/(data_train0.max() - data_train0.min())
X_train = data_train1.iloc[:, :-1]
y_train = data_train1.iloc[:, -1]

X_test = (data_test - data_test.min()) / (data_test.max() - data_test.min())

X_train = X_train.dropna(how='all')
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

print('shape of train:', X_train.shape)
print('shape of test:', X_test.shape)

start = time.time()
lr_clf = LogisticRegression(random_state=0, solver='lbfgs',
                            multi_class='multinomial', class_weight={0:1,1:1})
lr_clf.fit(X_train, y_train)
y_pred = lr_clf.predict_proba(X_test)
end = time.time()
print('total time:%2f' % (end - start))
result = pd.DataFrame(index=X_test.index - 40000)
result['Tag'] = y_pred[:, 1]
print('shape of result:', result.shape)
result.to_csv('../python_env01/data/new/sublimt.csv')


shape of train: (31179, 142)
shape of test: (31198, 142)
total time:2.481582
shape of result: (31198, 1)


In [43]:
result.head()

,Tag
UID,
30000,0.117106
30001,0.001757
30002,0.061371
30003,0.065355
30004,0.378243
